# Part 1 - Scraping

In [1]:
#Dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import time
import requests
import pprint 
from IPython.display import Markdown, display
import pymongo
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/Sally/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


In [4]:
# connect to NASA Mars news Site
url = 'https://redplanetscience.com/'
# Retrieve page with the requests module
browser.visit(url)

In [5]:
response = requests.get(url)
response

<Response [200]>

### NASA Mars News
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [6]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

# Examine the results, determine elements that contains sought info.
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet">
   <title>
    News - Mars Exploration Program
   </title>
  </link>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <s

In [11]:
latest_news = soup.find("li", class_="slide")
news_title = latest_news.find("content_title").text
news_p = latest_news.find(class_="article_teaser_body").text

print(f'------------------------------------------------')
print(f'TITLE: {news_title}')
print(f'------------------------------------------------')
print(f'PARAGRAPH: {news_p}')

AttributeError: 'NoneType' object has no attribute 'find'

### JPL Mars Space Images - Featured Image

In [12]:
# Visit the url for JPL Featured Space Image
# Set URL
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [13]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image 
featured_image = browser.find_by_id("full_image")
featured_image.click()

ElementDoesNotExist: no elements could be found with id "full_image"

In [14]:
# Need more info to find image url
browser.is_element_present_by_text("more info", wait_time=1)
more_info_button = browser.links.find_by_partial_text("more info")
more_info_button.click()

ElementDoesNotExist: no elements could be found with link by partial text "more info"

In [15]:
# Parse Results HTML with BeautifulSoup
html = browser.html
image_soup = BeautifulSoup(html, 'html.parser')

In [16]:
# Assign the url string to a variable called `featured_image_url`
# Make sure to find the image url to the full size `.jpg` image
featured_image_url = image_soup.select_one("figure.lede a img").get("src")
featured_image_url

AttributeError: 'NoneType' object has no attribute 'get'

In [17]:
# Make sure to save a complete url string for this image
featured_image_url = f"https://www.jpl.nasa.gov{featured_image_url}"
print(f"Complete URL String:\n{featured_image_url}")

NameError: name 'featured_image_url' is not defined

### Mars Facts

In [18]:
# Visit the Mars Facts webpage
# Set URL
url = 'https://galaxyfacts-mars.com/'
browser.visit(url)

In [19]:
# Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
mars_facts_df = pd.read_html("https://space-facts.com/mars/")[0]
print(mars_facts_df)

                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


In [20]:
# Clean up DataFrame, set index
mars_facts_df.columns=["Planet Profile", "Value"]
mars_facts_df.set_index("Planet Profile", inplace=True)
mars_facts_df

,Value
Planet Profile,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [21]:
# Visit the Astrogeology site
# Set URL
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [22]:
# Parse Results HTML with BeautifulSoup
html = browser.html
mars_weather_soup = BeautifulSoup(html, "html.parser")

In [23]:
# Save both the image url string for the full resolution hemisphere image, 
# and the Hemisphere title containing the hemisphere name
hemisphere_img_urls = []

links = browser.find_by_css("a.product-item h3")
for item in range(len(links)):
    hemisphere = {}
    
    browser.find_by_css("a.product-item h3")[item].click()
    
    sample_img = browser.links.find_by_text("Sample").first
    
    hemisphere["title"] = browser.find_by_css("h2.title").text
    hemisphere["img_url"] = sample_img["href"]     
    
    hemisphere_img_urls.append(hemisphere)
    
    browser.back()

In [24]:
# Use a Python dictionary to store the data using the keys `img_url` and `title`
hemisphere_img_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]